In [3]:
import pandas as pd
import numpy as np
import os
import sklearn
import xgboost as xgb
import sys
import matplotlib.pyplot as plt

## load trained synthesizer and generate X'

In [4]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from synthesizer import *

synthesizer_file_name = 'D:\SummerResearch\sdv trained model\\adult\\adult_synthesizer_onlyX.pkl'
synthesizer = load_synthesizer(synthesizer_file_name)

In [16]:
df_train = pd.read_csv('D:\SummerResearch\data\\adult\\adult_train.csv')
df_test = pd.read_csv('D:\SummerResearch\data\\adult\\adult_test.csv')

print(df_train.shape)
print(df_test.shape)
df_train.head()


(38095, 15)
(9526, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,37,Private,193106,Bachelors,13,Never-married,Sales,Not-in-family,White,Female,0,0,30,United-States,0
1,56,Self-emp-inc,216636,12th,8,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1651,40,United-States,0
2,53,Private,126977,HS-grad,9,Separated,Craft-repair,Not-in-family,White,Male,0,0,35,United-States,0
3,72,Private,205343,11th,7,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
4,46,State-gov,106705,Masters,14,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,38,United-States,0


In [23]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))
from commons.onehot import onehot

cat_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race',
            'sex', 'native-country']


ytrain = df_train['income']
ytest = df_test['income']

Xtrain_oh, Xtest_oh = onehot(df_train.drop('income', axis=1), df_test.drop('income', axis=1), cat_cols)
print(Xtrain_oh.head())

X_prime_xgb = synthesizer.sample(100000)
_, X_prime_xgb_oh = onehot(df_train.drop('income', axis=1), X_prime_xgb, cat_cols)
print(X_prime_xgb_oh.head())



   fnlwgt  capital-loss  education-num  age  hours-per-week  capital-gain  \
0  193106             0             13   37              30             0   
1  216636          1651              8   56              40             0   
2  126977             0              9   53              35             0   
3  205343             0              7   72              40             0   
4  106705             0             14   46              38             0   

   workclass_?  workclass_Federal-gov  workclass_Local-gov  \
0          0.0                    0.0                  0.0   
1          0.0                    0.0                  0.0   
2          0.0                    0.0                  0.0   
3          0.0                    0.0                  0.0   
4          0.0                    0.0                  0.0   

   workclass_Never-worked  ...  native-country_Portugal  \
0                     0.0  ...                      0.0   
1                     0.0  ...                

## XGBOOST

In [24]:
xgboost = xgb.XGBClassifier()

xgboost.fit(Xtrain_oh, ytrain)
print("Train score:")
print(xgboost.score(Xtrain_oh, ytrain))
print("Test score:")
print(xgboost.score(Xtest_oh, ytest))


Train score:
0.8943168394802468
Test score:
0.8731891664917069


In [26]:
y_prime_xgb = xgboost.predict(X_prime_xgb_oh)
df_prime_xgb = pd.DataFrame(pd.concat([X_prime_xgb_oh, pd.DataFrame(y_prime_xgb, columns=['income'])], axis=1))

df_prime_xgb.head()

,fnlwgt,capital-loss,education-num,age,hours-per-week,capital-gain,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,income
0,210786,0,7,24,32,10,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,115987,1,4,29,7,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,109130,0,9,24,50,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
3,175516,0,9,35,40,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,409291,1,10,44,40,10,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0


In [27]:
df_prime_xgb['income'].value_counts()

income
0    85747
1    14253
Name: count, dtype: int64

In [28]:
df_prime_xgb.to_csv('D:\SummerResearch\data\\adult\onehot_adult_sdv_xgb_100k.csv', index=False)

## Random Forest

In [29]:
rf = sklearn.ensemble.RandomForestClassifier()
rf.fit(Xtrain_oh, ytrain)
print("Train score:")
print(rf.score(Xtrain_oh, ytrain))
print("Test score:")
print(rf.score(Xtrain_oh, ytrain))

Train score:
0.9998162488515553
Test score:
0.9998162488515553


In [30]:
X_prime_rf = synthesizer.sample(100000)


sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))
from commons.onehot import onehot

cat_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race',
            'sex', 'native-country']


# ytrain = df_train['income']
# ytest = df_test['income']

# Xtrain_oh, Xtest_oh = onehot(df_train.drop('income', axis=1), df_test.drop('income', axis=1), cat_cols)
# print(Xtrain_oh.head())

_, X_prime_rf_oh = onehot(df_train.drop('income', axis=1), X_prime_rf, cat_cols)
print(X_prime_rf_oh.head())





y_prime_rf = rf.predict(X_prime_rf_oh)
df_prime_rf = pd.DataFrame(pd.concat([X_prime_rf_oh, pd.DataFrame(y_prime_rf, columns=['income'])], axis=1))
df_prime_rf.head()


   fnlwgt  capital-loss  education-num  age  hours-per-week  capital-gain  \
0  195216             0              9   42              20             0   
1  101007             0              9   37              50             6   
2  227901             0             16   71              60            11   
3   49737             1              7   18              40             4   
4  125173             1              9   48              59            10   

   workclass_?  workclass_Federal-gov  workclass_Local-gov  \
0          0.0                    0.0                  0.0   
1          0.0                    1.0                  0.0   
2          0.0                    0.0                  0.0   
3          0.0                    0.0                  0.0   
4          0.0                    0.0                  0.0   

   workclass_Never-worked  ...  native-country_Portugal  \
0                     0.0  ...                      0.0   
1                     0.0  ...                

,fnlwgt,capital-loss,education-num,age,hours-per-week,capital-gain,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,income
0,195216,0,9,42,20,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,101007,0,9,37,50,6,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
2,227901,0,16,71,60,11,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
3,49737,1,7,18,40,4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
4,125173,1,9,48,59,10,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0


In [31]:
df_prime_rf.to_csv('D:\SummerResearch\data\\adult\onehot_adult_sdv_rf_100k.csv', index=False)